In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# specify parameters
pipeline_params={
}
step_params={
}
substep_params={
    "SEED"         : 42, 
}

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(    
    tmp_inputs =
    [
        { ENTITY_NAME: "train_eval_config" },
        { ENTITY_NAME: "pretrain_weights" },
        { ENTITY_NAME: "yolox_obj_detector" }
    ],    
    outputs = 
    [
        { ENTITY_NAME: "yolox_obj_detector"}
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

![interface 2_train.drawio](./imgs/2_train.drawio.png)

In [ ]:
import logging
import os.path as osp
import os
logging.root.setLevel(substep_params.get('loggingLevel', 'INFO'))
logging.debug('Debug Output')

### Training model

#### Initializing modules from mmdetection, mmcv

In [ ]:
import torch, torchvision

print(f"{torch.__version__=}")
print(f"{torch.cuda.is_available()=}")
print(f"{torchvision.__version__=}")

if torch.cuda.is_available():
    device_id = torch.cuda.current_device()
    device_name = torch.cuda.get_device_name(device_id)
    print(f"{device_name=}")
    print(f"{torch.cuda.device_count()=}")

In [ ]:
from mmdet.datasets import PIPELINES

try:
    @PIPELINES.register_module()
    class DataAsList:
        def __call__(self, results):
            aug_data_dict = {key: [val] for key, val in results.items()}
            return aug_data_dict
except Exception as e:
    print(e)

In [ ]:
import copy
import os
import time
import warnings
import math

import mmcv
from mmcv import Config, ConfigDict

import mmdet
from mmdet.apis import init_random_seed, set_random_seed, train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.utils import get_root_logger

print(f"{mmcv.__version__=}")
print(f"{mmdet.__version__=}")

In [ ]:
import json

tmp_inputs = substep.tmp_inputs()

config_fn = os.path.join(tmp_inputs.train_eval_config, 'config.json')

with open(config_fn) as f_id:
    CONFIG = json.load(f_id)

In [ ]:
CONFIG

In [ ]:
cfg = Config.fromfile(CONFIG['config_file'])

In [ ]:
with open(osp.join(cfg.work_dir, 'config.json'), 'w') as f_id:
    json.dump(CONFIG, f_id, indent=4)

In [ ]:
# init the logger before other steps
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
log_file = osp.join(cfg.work_dir, f'latest.log')
logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

In [ ]:
base_seed = substep_params['SEED']**2
deterministic = False

meta = dict()
meta['config'] = cfg.pretty_text

# set random seeds
seed = init_random_seed(base_seed)
# logger.info(f'Set random seed to {seed}, '
#             f'deterministic: {deterministic}')
set_random_seed(seed, deterministic=deterministic)
cfg['seed'] = seed
meta['seed'] = seed
meta['exp_name'] = CONFIG['config_file']

In [ ]:
cfg.model

#### Initializing the model based on pretrain weights

In [ ]:
model = build_detector(cfg.model)

model.init_weights()

In [ ]:
datasets = [build_dataset(cfg.data.train)]

if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    datasets.append(build_dataset(val_dataset))
    
for i in range(len(datasets)):
    try:
        datasets[i].update_skip_type_keys
    except AttributeError:
        datasets[i].update_skip_type_keys = lambda x: x 

In [ ]:
model.CLASSES = datasets[0].CLASSES
datasets[0].CLASSES

In [ ]:
model.backbone.frozen_stages

#### Start model training 

In [ ]:
 train_detector(
    model,
    datasets,
    cfg,
    validate=True,
    timestamp=timestamp,
    meta=meta)

#### Copying an image from a validation dataset

In [ ]:
val_dataset.ann_file

In [ ]:
val_dataset

In [ ]:
# save one example image from eval_dataset
import json
from pathlib import Path
import shutil 

def load_json(json_file):
    with open(json_file) as io:
        json_data = json.load(io)
    return json_data

val_coco = load_json(val_dataset.ann_file)
assert val_coco
select_file = osp.join(val_dataset.img_prefix, val_coco["images"][0]["file_name"])
assert osp.exists(select_file)

shutil.copy(select_file, osp.join(CONFIG['work_dir'], f"test{Path(select_file).suffix}"))

### Copying a trained model
(weights, config, test image) in hdfs for subsequent transfer to other components

Since during the training process intermediate weights of the neural network can be created (for example, for epochs 10, 20, 30, etc.)
then it doesn't make much sense to copy all the intermediate files to another step in the pipeline.
Therefore, we will copy the weights and the necessary configs into a separate directory and we will copy these files to outputs

In [ ]:
!ls -lah {cfg.work_dir}

In [ ]:
cfg.work_dir

In [ ]:
# copy selected files to another dir
import shutil 
import os.path as osp

weights_dir = osp.join(cfg.work_dir, "weights")
os.makedirs(weights_dir, exist_ok=True)

model_path = cfg.work_dir
files = [osp.join(model_path, file) for file in os.listdir(model_path)]
models_pth  = [file for file in files if '.pth' in file if osp.isfile(file)]
best_models = [file for file in models_pth if 'best' in file]
latest_models = [file for file in models_pth if 'latest' in file]

shutil.copy(osp.join(cfg.work_dir, "config.json"), osp.join(weights_dir, "config.json"))
shutil.copy(osp.join(cfg.work_dir, f"test{Path(select_file).suffix}"), osp.join(weights_dir, f"test{Path(select_file).suffix}"))
shutil.copy(cfg.filename, osp.join(weights_dir, osp.basename(cfg.filename)))

for fpath in latest_models:
    shutil.copy(fpath, fpath.replace(model_path, weights_dir))
for fpath in best_models:
    shutil.copy(fpath, fpath.replace(model_path, weights_dir))

In [ ]:
!ls -lah {weights_dir}

#### delete information about use cache

In [ ]:
cfg = Config.fromfile(osp.join(weights_dir, "last_cfg.py"))
cfg.load_from = ""
cfg.train_dataset.ann_file = ""
cfg.test_dataset.ann_file = ""
cfg.data.train.ann_file = ""
cfg.data.val.ann_file = ""
cfg.data.test.ann_file = ""
cfg.work_dir = "" 
config_file = osp.join(cfg.work_dir, "last_cfg.py")
cfg.dump(file=osp.join(weights_dir, "last_cfg.py"))

In [ ]:
with open(osp.join(weights_dir, "config.json")) as f_id:
    temp_CONFIG = json.load(f_id)
temp_CONFIG.pop("eval_datasets") if "eval_datasets" in temp_CONFIG else ""
temp_CONFIG.pop("train_datasets") if "train_datasets" in temp_CONFIG else ""
temp_CONFIG.pop("train_output_file") if "train_output_file" in temp_CONFIG else ""
temp_CONFIG.pop("eval_output_file") if "eval_output_file" in temp_CONFIG else ""
temp_CONFIG.pop("work_dir") if "work_dir" in temp_CONFIG else ""
temp_CONFIG["config_file"] = osp.basename(temp_CONFIG["config_file"])

with open(osp.join(weights_dir, "config.json"), 'w') as f_id:
    json.dump(temp_CONFIG, f_id, indent=4)

### Send training model to outputs

In [ ]:
from sinara.store import SinaraStore
outputs = substep.outputs()
SinaraStore.archive_tmp_files_to_store(tmp_dir=weights_dir, store_path=outputs.yolox_obj_detector)